#Desafio técnico MINEHR

### Por: Katia Cardoso

## Importação de diferentes bases

In [0]:
import pandas as pd
import os

# URL do arquivo CSV
url_csv = 'https://storage.googleapis.com/desafio-ed/ingestion/base_colaboradores_2022.csv'
# URL do arquivo Excel (.xlsx)
url_excel = 'https://storage.googleapis.com/desafio-ed/ingestion/base_colaboradores_2023.xlsx'

# Importando dados do Excel Online
df_xlsx = pd.read_excel(url_excel)
# Importando dados do CSV com codificação "latin-1" e separador ";"
df_csv = pd.read_csv(url_csv, encoding='latin-1', sep=';')

## Padronização dos nomes das colunas

In [0]:
# Imprimir os nomes das colunas dos DataFrames
print("Nomes das colunas de df_csv antes dos ajustes:")
print(df_csv.columns)

print("\nNomes das colunas de df_excel antes dos ajustes:")
print(df_xlsx.columns)



Nomes das colunas de df_csv antes dos ajustes:
Index(['Matrícula    ', 'MÊS REFERENCIA', 'FUNCIONARIO', 'DATA ADMISSAO',
       'NACIONALIDADE', 'DATA DEMISSAO', 'ESTADO CIVIL', 'SEXO', 'CARGO',
       'EMPRESA', 'DATA NASCIMENTO'],
      dtype='object')

Nomes das colunas de df_excel antes dos ajustes:
Index(['mês referência', 'matricula', 'data admissao', 'funcionario',
       'nacionalidade', 'estado civil', 'sexo', 'cargo', 'empresa',
       'data de nascimento', 'data demissao'],
      dtype='object')


In [0]:
from unidecode import unidecode

# Função para normalizar nomes de colunas
def normalize_column_names(df):
    # Normalizar nomes das colunas (acentos, minúsculas e espaços)
    normalized_columns = [unidecode(col).lower().strip().replace(' ', '_') for col in df.columns]
    df.columns = normalized_columns
    return df

# Normalizar nomes das colunas para ambos os DataFrames
df_csv = normalize_column_names(df_csv)
df_xlsx = normalize_column_names(df_xlsx)

# Modificar o nome da coluna "data_de_nascimento" no DataFrame Excel
df_xlsx.rename(columns={'data_de_nascimento': 'data_nascimento'}, inplace=True)

# Verificar se os nomes das colunas são idênticos
if df_csv.columns.equals(df_xlsx.columns):
    # Se os nomes das colunas são idênticos, reorganize as colunas
    df_csv = df_csv[df_xlsx.columns]
else:
    # Imprimir as colunas que são diferentes
    different_columns = [col for col in df_csv.columns if col not in df_xlsx.columns]
    print(f"Colunas diferentes nos DataFrames: {different_columns}")




Colunas diferentes nos DataFrames: []


In [0]:
# Imprimir os nomes das colunas dos DataFrames
print("Nomes das colunas de df_csv apos ajustes:")
print(df_csv.columns)

print("\nNomes das colunas de df_excel apos ajustes:")
print(df_xlsx.columns)



Nomes das colunas de df_csv apos ajustes:
Index(['matricula', 'mes_referencia', 'funcionario', 'data_admissao',
       'nacionalidade', 'data_demissao', 'estado_civil', 'sexo', 'cargo',
       'empresa', 'data_nascimento'],
      dtype='object')

Nomes das colunas de df_excel apos ajustes:
Index(['mes_referencia', 'matricula', 'data_admissao', 'funcionario',
       'nacionalidade', 'estado_civil', 'sexo', 'cargo', 'empresa',
       'data_nascimento', 'data_demissao'],
      dtype='object')


## União dos arquivos

In [0]:
# Concatenar os DataFrames por linhas (um abaixo do outro)
df_final = pd.concat([df_csv, df_xlsx], ignore_index=True)

# Salvando o DataFrame como um arquivo CSV
df_final.to_csv('df_final.csv', index=False, encoding='utf-8')

# Visualizando as primeiras 5 linhas do DataFrame
df_final.head()

,matricula,mes_referencia,funcionario,data_admissao,nacionalidade,data_demissao,estado_civil,sexo,cargo,empresa,data_nascimento
0,27,28/02/2022,Colaborador 27,05/12/2005,brasil,NaN,Casado,Masculino,Eletr Lv Transmissao Iii,Empresa 6,01/01/1977
1,28,31/01/2022,Colaborador 28,14/04/2003,brasil,NaN,Solteiro,Feminino,Coord Tesouraria,Empresa 5,01/04/1979
2,28,28/02/2022,Colaborador 28,14/04/2003,brasil,NaN,Casado,Feminino,Coord Tesouraria,Empresa 5,01/04/1979
3,28,31/03/2022,Colaborador 28,14/04/2003,brasil,NaN,Solteiro,Feminino,Coord Tesouraria,Empresa 2,01/04/1979
4,28,30/04/2022,Colaborador 28,14/04/2003,brasil,NaN,Solteiro,Feminino,Coord Tesouraria,Empresa 2,01/04/1979


In [0]:
#Verificação do numero de linhas e colunas arquivo .csv
num_linhas_csv, num_colunas_csv = df_csv.shape
print(f'Número de linhas csv : {num_linhas_csv}')
print(f'Número de colunas csv: {num_colunas_csv} \n')


#Verificação do numero de linhas e colunas arquivo .xlsx
num_linhas_xlsx, num_colunas_xlsx = df_xlsx.shape
print(f'Número de linhas xlsx: {num_linhas_xlsx}')
print(f'Número de colunas xlsx : {num_colunas_xlsx} \n ')

#Verificação do numero de linhas e colunas arquivo final
num_linhas, num_colunas = df_final.shape
print(f'Número de linhas final: {num_linhas}')
print(f'Número de colunas final: {num_colunas}')


Número de linhas csv : 20054
Número de colunas csv: 11 

Número de linhas xlsx: 27297
Número de colunas xlsx : 11 
 
Número de linhas final: 47351
Número de colunas final: 11


## Conversão para um DataFrame Spark

In [0]:
# Verificação dos tipos de dados das colunas
print(df_final.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47351 entries, 0 to 47350
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   matricula        47351 non-null  int64 
 1   mes_referencia   47351 non-null  object
 2   funcionario      47351 non-null  object
 3   data_admissao    47351 non-null  object
 4   nacionalidade    47346 non-null  object
 5   data_demissao    436 non-null    object
 6   estado_civil     47116 non-null  object
 7   sexo             47351 non-null  object
 8   cargo            47351 non-null  object
 9   empresa          47351 non-null  object
 10  data_nascimento  47351 non-null  object
dtypes: int64(1), object(10)
memory usage: 4.0+ MB
None


In [0]:
import pyspark.sql.types as t

# Conversão do tipo de dados das colunas com o tipo 'object' para 'str'
for col in df_final.columns:
    if df_final[col].dtype == 'object':
        df_final[col] = df_final[col].astype(str)

In [0]:
from pyspark.sql import SparkSession

# Conversão do DataFrame do Pandas para um DataFrame do Spark
spark_df = spark.createDataFrame(df_final)

# Exiba o esquema do DataFrame Spark
spark_df.printSchema()

root
 |-- matricula: long (nullable = true)
 |-- mes_referencia: string (nullable = true)
 |-- funcionario: string (nullable = true)
 |-- data_admissao: string (nullable = true)
 |-- nacionalidade: string (nullable = true)
 |-- data_demissao: string (nullable = true)
 |-- estado_civil: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- cargo: string (nullable = true)
 |-- empresa: string (nullable = true)
 |-- data_nascimento: string (nullable = true)



In [0]:
# Exiba o DataFrame ajustado
spark_df.show()

+---------+--------------+--------------+-------------+-------------+-------------+------------+---------+--------------------+---------+---------------+
|matricula|mes_referencia|   funcionario|data_admissao|nacionalidade|data_demissao|estado_civil|     sexo|               cargo|  empresa|data_nascimento|
+---------+--------------+--------------+-------------+-------------+-------------+------------+---------+--------------------+---------+---------------+
|       27|    28/02/2022|Colaborador 27|   05/12/2005|     brasil  |          nan|      Casado|Masculino|Eletr Lv Transmis...|Empresa 6|     01/01/1977|
|       28|    31/01/2022|Colaborador 28|   14/04/2003|     brasil  |          nan|    Solteiro| Feminino|    Coord Tesouraria|Empresa 5|     01/04/1979|
|       28|    28/02/2022|Colaborador 28|   14/04/2003|     brasil  |          nan|      Casado| Feminino|    Coord Tesouraria|Empresa 5|     01/04/1979|
|       28|    31/03/2022|Colaborador 28|   14/04/2003|     brasil  |       

In [0]:
# Verificação do tipo de dados do DataFrame
print(type(spark_df))

<class 'pyspark.sql.dataframe.DataFrame'>


In [0]:
# Imprime o esquema do DataFrame
print(spark_df.dtypes)

[('matricula', 'bigint'), ('mes_referencia', 'string'), ('funcionario', 'string'), ('data_admissao', 'string'), ('nacionalidade', 'string'), ('data_demissao', 'string'), ('estado_civil', 'string'), ('sexo', 'string'), ('cargo', 'string'), ('empresa', 'string'), ('data_nascimento', 'string')]


## Padronização dos dados


 - trim(col(coluna)) remove os espaços em branco no início e no final de cada string na coluna
 - initcap() coloca as iniciais das palavras em maiúsculas. 

In [0]:
from pyspark.sql.functions import trim, initcap, col

# Lista de todas as colunas do DataFrame
columns = spark_df.columns

# Lista de colunas textuais 
textual_columns = ['mes_referencia', 'funcionario', 'data_admissao', 'nacionalidade', 'data_demissao', 'estado_civil', 'sexo', 'cargo', 'empresa', 'data_nascimento']  

# Aplicação das transformações nas colunas textuais
for column in textual_columns:
    spark_df = spark_df.withColumn(column, initcap(trim(col(column))))

# Mostra o DataFrame após as transformações
spark_df.show()

+---------+--------------+--------------+-------------+-------------+-------------+------------+---------+--------------------+---------+---------------+
|matricula|mes_referencia|   funcionario|data_admissao|nacionalidade|data_demissao|estado_civil|     sexo|               cargo|  empresa|data_nascimento|
+---------+--------------+--------------+-------------+-------------+-------------+------------+---------+--------------------+---------+---------------+
|       27|    28/02/2022|Colaborador 27|   05/12/2005|       Brasil|          Nan|      Casado|Masculino|Eletr Lv Transmis...|Empresa 6|     01/01/1977|
|       28|    31/01/2022|Colaborador 28|   14/04/2003|       Brasil|          Nan|    Solteiro| Feminino|    Coord Tesouraria|Empresa 5|     01/04/1979|
|       28|    28/02/2022|Colaborador 28|   14/04/2003|       Brasil|          Nan|      Casado| Feminino|    Coord Tesouraria|Empresa 5|     01/04/1979|
|       28|    31/03/2022|Colaborador 28|   14/04/2003|       Brasil|       

verificar requisito: Para as colunas numéricas, certifique-se que tenha no máximo 4 casa decimais.

In [0]:
from pyspark.sql.functions import col, length

# Nome da coluna numérica 
numeric_columns = 'matricula'

# Verificação do comprimento dos valores na coluna numérica
spark_df.withColumn('comprimento', length(col(numeric_columns))).filter(col('comprimento') > 4).show()

+---------+--------------+-----------+-------------+-------------+-------------+------------+----+-----+-------+---------------+-----------+
|matricula|mes_referencia|funcionario|data_admissao|nacionalidade|data_demissao|estado_civil|sexo|cargo|empresa|data_nascimento|comprimento|
+---------+--------------+-----------+-------------+-------------+-------------+------------+----+-----+-------+---------------+-----------+
+---------+--------------+-----------+-------------+-------------+-------------+------------+----+-----+-------+---------------+-----------+

